<a href="https://colab.research.google.com/github/m-hironori/learing-keras/blob/master/2_2%E5%80%A4%E5%88%86%E9%A1%9E(IMDb%E3%83%AC%E3%83%93%E3%83%A5%E3%83%BC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.5'

# 2値分類(IMDbレビュー)

kerasによるモデル作成の基本として、2値分類の例をやってみましょう。
今回は、[IMDb](https://www.imdb.com/)という映画レビューサイトのポジネガ分類をやってみましょう。



 

## 問題設定

まずは、問題設定を確認してみましょう。

### 問題設定の確認


問題属性 | 問題内容
--- | ---
入力データ形式 | レビューテキスト（英語）
予測すること | レビュー内容が、ポジティブ or ネガティブ
評価指標 | accuracy
評価方法 | ホールドアウト

レビューテキストを入力に、ポジティブかネガティブかを当てることが問題設定です。

keras には、あらかじめこのデータセットが用意されていて、扱いやすいため、
keras のテキストを使った例としてはよく使われます。


## データ準備

データを準備してみましょう

### データ取得する

In [2]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 0s 0us/step


データロードの関数が用意されているので、これを呼ぶだけで、データ取得が終わります。

ダウンロード後、num_words=10000で、出現頻度順上位10,000の単語のみをロードしています。

### 内容を確認する

* テンソルで準備：データはテンソル ( Numpy でーたが扱いやすい ) として準備
* 値を正規化：値は [ -1 , -1 ] や [ 0 , 1 ] ぐらいの値にスケーリングしておく
* スケール合わせ：特徴量の値の範囲がことなったり、異なる種類のデータを使う場合は、スケールを合わせるためにデータを正規化する 

基本的に、上記3点がデータ準備で必要なことです。テンソル準備、値を正規化、そしてスケールを合わせる。

今回はテキストデータのみなので、スケール合わせは必要なさそうです。

### データ型を確認する

In [4]:
print("\tデータ型")
print(f"訓練データ\t{type(train_data)}")
print(f"訓練ラベル\t{type(train_labels)}")
print(f"テストデータ\t{type(test_data)}")
print(f"テストラベル\t{type(test_labels)}")

	データ型
訓練データ	<class 'numpy.ndarray'>
訓練ラベル	<class 'numpy.ndarray'>
テストデータ	<class 'numpy.ndarray'>
テストラベル	<class 'numpy.ndarray'>


Numpy形式で用意されているので、今回もデータ型はこのままでよさそうです。

In [5]:
import numpy as np

print("\tデータサイズ\t型\t最大値\t最小値")
print(f"訓練データ\t{train_data.shape}\t{train_data.dtype}\t{train_data.max()}\t{train_data.min()}")
print(f"訓練ラベル\t{train_labels.shape}\t{train_labels.dtype}\t{train_labels.max()}\t{train_labels.min()}")
print(f"テストデータ\t{test_data.shape}\t{test_data.dtype}\t{test_data.max()}\t{test_data.min()}")
print(f"テストラベル\t{test_labels.shape}\t{test_labels.dtype}\t{test_labels.max()}\t{test_labels.min()}")

	データサイズ	型	最大値	最小値
訓練データ	(25000,)	object	[1, 9995, 2, 7, 2208, 7335, 3135, 4173, 3783, 509, 1683, 4702, 2, 2, 6, 201, 7, 6415, 687, 2, 2, 7, 6387, 548, 139, 7583, 295, 34, 4485, 5302, 2, 2, 2, 7, 2, 2, 146, 24, 1017, 2282, 133, 21, 4, 1591, 3113, 786, 2, 16, 125, 4, 2, 9790, 2039, 137, 267, 2, 5, 2, 120, 2024, 980, 2, 1248, 5666, 727, 1405, 6879, 1060, 6442, 18, 2, 1461, 2, 1883, 445, 109, 5369, 3696, 33, 236, 786, 5580, 7994, 8, 2, 2, 103, 2, 98, 11, 2, 1461, 24, 66, 351, 1461, 165, 116, 17, 2545, 18, 6717, 8694, 5514, 980, 2545, 165, 734, 18, 2311, 52, 84, 157, 18, 7232, 11, 661, 8, 607, 3531, 223, 1066, 445, 9236, 996, 8, 2898, 4931, 8, 5526, 8316, 7880, 154, 378, 459, 18, 6942, 632, 5, 79, 2024, 18, 68, 4918, 813, 2456, 2, 17, 840, 4807, 3854, 23, 136, 159, 5770, 852, 1698, 632, 7708, 7993, 1003, 1372, 5747, 2, 7, 5840, 5408, 11, 7200, 120, 4682, 7, 2, 3368, 2103, 8, 1140, 2, 880, 1692, 7, 68, 205, 3565, 5903, 21, 3204, 1372, 2, 2, 1098, 125, 128, 2429, 21, 9000, 2, 632, 2, 245, 39,

よさそうに思いきや、データの内容が、object型でした。このままではだめそうですね。

訓練ラベルは、整数型で1と0の値しかないため、このままでよさそうです。

### データ内容を確認する

In [6]:
# 訓練データ内容を表示する
train_data[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

入力はテキストデータのはずなのに、数字の列になっています。




### テキストデータの前処理



No | 処理     | 内容 | 例(This movie was just brilliant casting)
---|----------|------|---
1  | Tokenize | テキストを単語に区切る | `["This", "movie", "was", "just", "brilliant", "casting"]`
2  | Normarize| 大文字を小文字に変換、活用形を原型に変えるなど | `["this", "movie", "was", "just", "brilliant", "casting"]`
3  | Indexing | 単語の辞書を作成しながらインデックスに変換 | `[14, 22, 16, 43, 530, 13, 19]`


テキストデータを扱うときは、各単語を単語のインデックスデータに変換して扱います。

まず、テキストを、単語に区切ります。これが、データの最小単位になります。
次に、正規化を行います。大文字を小文字に変換したり、活用形を原型に戻したり(played -> play)します。この処理は、実施するデータによって何を行うかまちまちです。
次に、単語の辞書を作成しながらインデックスに変換していきます。"this"はインデックス14に変換するといった辞書を作成していき、同じ単語は同じインデックス番号となるようにします。

このような処理によって単語インデックスのシーケンスができます。

この imdb データセットでは、あらかじめ、テキストデータをインデックスデータに変換してくれています。

### 単語インデックスデータをテキストに戻してみる

In [8]:
# 単語インデックス(単語->インデックス))を　逆転させる(インデックス -> 単語 )
reverse_word_index = dict([(value, key) for (key, value) in imdb.get_word_index().items()])
# imdbデータセットではインデックスオフセットは 3
# 0: パディング用に予約, 1: シーケンス開始を示す, 2: 足きりして知らない単語用.
decoded = []
for i in train_data[0]:
    if i == 0:
        decoded.add("[PAD]")
    if i == 1:
        decoded.add("[START]")
    if i == 2:
        decoded.add("[UNK]")
    if i
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])
decoded_review

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th